In [1]:
import pandas as pd
import numpy as np
import math
import random as python_random
import io
import os
import glob
import matplotlib.pyplot as plt
from numpy import linalg as LA
import matplotlib.pyplot as plt

from IPython.display import clear_output

seeds=[19,31,38,47,77]

In [2]:
diseases = ['Enlarged Cardiomediastinum','Cardiomegaly','Lung Opacity','Lung Lesion','Edema',
        'Consolidation','Pneumonia','Atelectasis','Pneumothorax','Pleural Effusion','Pleural Other',
        'Fracture','Support Devices','No Finding']

diseases_abbr = {'Cardiomegaly': 'Cardiomegaly',
                 'Effusion': 'Effusion',
                 'Enlarged Cardiomediastinum': 'Enlarged Card.',
                 'Lung Lesion': 'Lung Lesion',
                 'Atelectasis': 'Atelectasis',
                 'Pneumonia': 'Pneumonia',
                 'Pneumothorax': 'Pneumothorax',
                 'Consolidation': 'Consolidation',
                 'Edema': 'Edema',
                 'Pleural Effusion': 'Effusion',
                 'Pleural Other': 'Pleural Other',
                 'Fracture': 'Fracture',
                 'Support Devices': 'Sup. Devices',
                 'Lung Opacity': 'Air. Opacity',
                 'No Finding': 'No Finding'
                }


age_decile = ['60-80', '40-60', '20-40', '80+', '0-20']

gender = ['M', 'F']

race = ['WHITE', 'BLACK/AFRICAN AMERICAN',
        'HISPANIC/LATINO', 'OTHER', 'ASIAN',
        'AMERICAN INDIAN/ALASKA NATIVE']

factor = [gender, age_decile, race]

factor_str = ['gender', 'age_decile', 'race']


## TPR Disparities

In [3]:
def tpr(df, d, c, category_name):

  pred_disease = "bi_" + d
  gt = df.loc[(df[d] == 1) & (df[category_name] == c), :]
  pred = df.loc[(df[pred_disease] == 1) & (df[d] == 1) & (df[category_name] == c), :]

  if len(gt) != 0:
    TPR = len(pred) / len(gt)
    return TPR
  else:
    # print("Disease", d, "in category", c, "has zero division error")
    return np.NAN

In [4]:
def TPR_Disparities(df,diseases,category,category_name,seed=19,tpr_gaps_results_path_dir_dir="default"):

  plt.rcParams.update({'font.size': 18})
  GAP_total = []
  percentage_total = []
  cate = []

  if category_name == 'gender':
    Run1_sex = pd.DataFrame(diseases, columns=["diseases"])

  if category_name == 'age_decile':
    Run1_age = pd.DataFrame(diseases, columns=["diseases"])

  if category_name == 'race':
    Run1_race = pd.DataFrame(diseases, columns=["diseases"])


  for c in category:

    GAP_y = []
    percentage_y = []

    for d in diseases:

      pred_disease = "bi_" + d

      gt = df.loc[(df[d] == 1) & (df[category_name] == c), :]
      pred = df.loc[(df[pred_disease] == 1) & (df[d] == 1) & (df[category_name] == c), :]

      n_gt = df.loc[(df[d] == 1) & (df[category_name] != c) & (df[category_name] != 0), :]
      n_pred = df.loc[(df[pred_disease] == 1) & (df[d] == 1) & (df[category_name] != c) & (df[category_name] != 0), :]

      pi_gy = df.loc[(df[d] == 1) & (df[category_name] == c), :]
      pi_y = df.loc[(df[d] == 1) & (df[category_name] != 0), :]

      if len(gt) != 0  and len(n_gt) != 0 and len(pi_y) != 0:

        TPR = len(pred) / len(gt)

        n_TPR = len(n_pred) / len(n_gt)

        percentage = len(pi_gy) / len(pi_y)

        if category_name != 'gender':
          temp_TPR= []

          for c1 in category:
            _tpr = tpr(df, d, c1, category_name)

            if _tpr != -1:
              temp_TPR.append(_tpr)


          temp_TPR_Filtered= [x for x in temp_TPR if not math.isnan(x)]
          temp_TPR_Filtered.sort()


          if len(temp_TPR_Filtered) % 2 == 0:

            median = (temp_TPR_Filtered[(len(temp_TPR_Filtered) // 2) - 1]
                      + temp_TPR_Filtered[(len(temp_TPR_Filtered) // 2)])/2

          else:
            median = temp_TPR_Filtered[(len(temp_TPR_Filtered) // 2)]

          GAP=TPR - median

        else:

          GAP = TPR - n_TPR

        """ This portion of code is used for debuging purpose only """
        # if category_name=='age_decile' and c=='60-80' and d=='Fracture':
        #   print(f'Current category : {c}')
        #   print(f'Current disease : {d}')
        #   pdb.set_trace()

        GAP_y.append(GAP)
        percentage_y.append(percentage)

      else:

        GAP_y.append(np.NAN)
        percentage_y.append(0)


    GAP_total.append(GAP_y)
    percentage_total.append(percentage_y)

    c = c.replace(' ', '_', 3)
    c = c.replace('/', '_', 3)
    cate.append(c)


  GAP_total = np.array(GAP_total)

  if category_name == 'age_decile':
    print(f'GAP_total: {GAP_total}')

  # Create a new array of x-values for the non-NaN diseases
  x = np.arange(len(diseases))

  print("len(GAP_total): ",len(GAP_total))
  for i in range(len(GAP_total)):

    if category_name=='age_decile':

      if i==0:

        Percent0 = pd.DataFrame(percentage_total[i], columns=["%60-80"])
        Run1_age = pd.concat([Run1_age, Percent0.reindex(Run1_age.index)], axis=1)

        Gap0 = pd.DataFrame(GAP_total[i], columns=["Gap_60-80"])
        Run1_age = pd.concat([Run1_age, Gap0.reindex(Run1_age.index)], axis=1)

      if i==1:

        Percent1 = pd.DataFrame(percentage_total[i], columns=["%40-60"])
        Run1_age = pd.concat([Run1_age, Percent1.reindex(Run1_age.index)], axis=1)

        Gap1 = pd.DataFrame(GAP_total[i], columns=["Gap_40-60"])
        Run1_age = pd.concat([Run1_age, Gap1.reindex(Run1_age.index)], axis=1)

      if i == 2:

        Percent2 = pd.DataFrame(percentage_total[i], columns=["%20-40"])
        Run1_age = pd.concat([Run1_age, Percent2.reindex(Run1_age.index)], axis=1)

        Gap2 = pd.DataFrame(GAP_total[i], columns=["Gap_20-40"])
        Run1_age = pd.concat([Run1_age, Gap2.reindex(Run1_age.index)], axis=1)

      if i == 3:

        Percent3 = pd.DataFrame(percentage_total[i], columns=["%80+"])
        Run1_age = pd.concat([Run1_age, Percent3.reindex(Run1_age.index)], axis=1)

        Gap3 = pd.DataFrame(GAP_total[i], columns=["Gap_80+"])
        Run1_age = pd.concat([Run1_age, Gap3.reindex(Run1_age.index)], axis=1)

      if i == 4:

        Percent4 = pd.DataFrame(percentage_total[i], columns=["%0-20"])
        Run1_age = pd.concat([Run1_age, Percent4.reindex(Run1_age.index)], axis=1)

        Gap4 = pd.DataFrame(GAP_total[i], columns=["Gap_0-20"])
        Run1_age = pd.concat([Run1_age, Gap4.reindex(Run1_age.index)], axis=1)

      Run1_age.to_csv(tpr_gaps_results_path_dir_dir+"Run_seed"+str(seed)+"_TPR_GAP_Age.csv")

    if category_name =='gender':

      if i == 0:

        MalePercent = pd.DataFrame(percentage_total[i], columns=["%M"])
        Run1_sex = pd.concat([Run1_sex, MalePercent.reindex(Run1_sex.index)], axis=1)

        MaleGap = pd.DataFrame(GAP_total[i], columns=["Gap_M"])
        Run1_sex = pd.concat([Run1_sex, MaleGap.reindex(Run1_sex.index)], axis=1)

      else:

        FeMalePercent = pd.DataFrame(percentage_total[i], columns=["%F"])
        Run1_sex = pd.concat([Run1_sex, FeMalePercent.reindex(Run1_sex.index)], axis=1)

        FeMaleGap = pd.DataFrame(GAP_total[i], columns=["Gap_F"])
        Run1_sex = pd.concat([Run1_sex, FeMaleGap.reindex(Run1_sex.index)], axis=1)


      Run1_sex.to_csv(tpr_gaps_results_path_dir_dir+"Run_seed"+str(seed)+"_TPR_GAP_sex.csv")

    if category_name=='race':

      if i==0:

        Percent_White = pd.DataFrame(percentage_total[i], columns=["%White"])
        Run1_race = pd.concat([Run1_race, Percent_White.reindex(Run1_race.index)], axis=1)

        Gap_White= pd.DataFrame(GAP_total[i], columns=["Gap_White"])
        Run1_race = pd.concat([Run1_race, Gap_White.reindex(Run1_race.index)], axis=1)

      if i==1:

        Percent_Black= pd.DataFrame(percentage_total[i], columns=["%Black"])
        Run1_race = pd.concat([Run1_race, Percent_Black.reindex(Run1_race.index)], axis=1)

        Gap_Black = pd.DataFrame(GAP_total[i], columns=["Gap_Black"])
        Run1_race = pd.concat([Run1_race, Gap_Black.reindex(Run1_race.index)], axis=1)

      if i == 2:

        Percent_Hisp= pd.DataFrame(percentage_total[i], columns=["%Hisp"])
        Run1_race = pd.concat([Run1_race, Percent_Hisp.reindex(Run1_race.index)], axis=1)

        Gap_Hisp= pd.DataFrame(GAP_total[i], columns=["Gap_Hisp"])
        Run1_race = pd.concat([Run1_race, Gap_Hisp.reindex(Run1_race.index)], axis=1)

      if i == 3:

        Percent_Other = pd.DataFrame(percentage_total[i], columns=["%Other"])
        Run1_race = pd.concat([Run1_race, Percent_Other.reindex(Run1_race.index)], axis=1)

        Gap_Other= pd.DataFrame(GAP_total[i], columns=["Gap_Other"])
        Run1_race = pd.concat([Run1_race, Gap_Other.reindex(Run1_race.index)], axis=1)


      if i == 4:

        Percent_Asian = pd.DataFrame(percentage_total[i], columns=["%Asian"])
        Run1_race = pd.concat([Run1_race, Percent_Asian.reindex(Run1_race.index)], axis=1)

        Gap_Asian= pd.DataFrame(GAP_total[i], columns=["Gap_Asian"])
        Run1_race = pd.concat([Run1_race, Gap_Asian.reindex(Run1_race.index)], axis=1)

      if i == 5:

        Percent_American = pd.DataFrame(percentage_total[i], columns=["%American"])
        Run1_race = pd.concat([Run1_race, Percent_American.reindex(Run1_race.index)], axis=1)

        Gap_American= pd.DataFrame(GAP_total[i], columns=["Gap_American"])
        Run1_race = pd.concat([Run1_race, Gap_American.reindex(Run1_race.index)], axis=1)

      Run1_race.to_csv(tpr_gaps_results_path_dir_dir+"Run_seed"+str(seed)+"_TPR_GAP_race.csv")


In [6]:
for seed in seeds:

    np.random.seed(seed)
    python_random.seed(seed)
    
    base_path = "./Prediction_results/"
    
    # Create directory model weightes saving

    tpr_gaps_results_path_dir_dir='./TPR_GAPS/'
    os.makedirs(os.path.dirname(tpr_gaps_results_path_dir_dir), exist_ok=True)

    df = pd.read_csv(f"{base_path}bipred_{seed}.csv")
    
    ''' TPR Disparities '''

    for i in range(len(factor)):
        TPR_Disparities(df,diseases,factor[i],factor_str[i],seed,tpr_gaps_results_path_dir_dir)
    
    print(f'Seed : {seed}')

C:\Users\gebre\AppData\Local\Temp\ipykernel_16812\2086752001.py:13: DtypeWarning: Columns (1,3,6,7,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{base_path}bipred_{seed}.csv")


len(GAP_total):  2
GAP_total: [[ 0.02804029  0.01897696  0.00387182  0.          0.0215358   0.01821331
   0.          0.01705278 -0.03661454  0.00360846  0.03818994  0.
   0.0046906  -0.087661  ]
 [ 0.         -0.00171406 -0.01531189  0.00306577  0.          0.
   0.02806835  0.          0.00263027  0.          0.04889882  0.00192993
   0.          0.        ]
 [-0.01849264 -0.01752791 -0.03099402  0.04269281 -0.03381096 -0.00333708
   0.06631837 -0.01851874  0.         -0.01790068  0.          0.03307649
  -0.0129928   0.03337618]
 [ 0.04116849  0.02752705  0.         -0.02471776  0.03815551 -0.00716548
  -0.10258822  0.00628646 -0.19573792  0.00325833 -0.09143969 -0.07710009
  -0.01368186 -0.21948096]
 [-0.00772148  0.          0.00761749 -0.13405797 -0.07810548  0.01536754
  -0.03579093 -0.07200972  0.15137835 -0.02668517 -0.05810636 -0.01368068
   0.06473829  0.06142496]]
len(GAP_total):  5
len(GAP_total):  6
Seed : 19


C:\Users\gebre\AppData\Local\Temp\ipykernel_16812\2086752001.py:13: DtypeWarning: Columns (1,3,6,7,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{base_path}bipred_{seed}.csv")


len(GAP_total):  2
GAP_total: [[ 0.02804029  0.01897696  0.00387182  0.          0.0215358   0.01821331
   0.          0.01705278 -0.03661454  0.00360846  0.03818994  0.
   0.0046906  -0.087661  ]
 [ 0.         -0.00171406 -0.01531189  0.00306577  0.          0.
   0.02806835  0.          0.00263027  0.          0.04889882  0.00192993
   0.          0.        ]
 [-0.01849264 -0.01752791 -0.03099402  0.04269281 -0.03381096 -0.00333708
   0.06631837 -0.01851874  0.         -0.01790068  0.          0.03307649
  -0.0129928   0.03337618]
 [ 0.04116849  0.02752705  0.         -0.02471776  0.03815551 -0.00716548
  -0.10258822  0.00628646 -0.19573792  0.00325833 -0.09143969 -0.07710009
  -0.01368186 -0.21948096]
 [-0.00772148  0.          0.00761749 -0.13405797 -0.07810548  0.01536754
  -0.03579093 -0.07200972  0.15137835 -0.02668517 -0.05810636 -0.01368068
   0.06473829  0.06142496]]
len(GAP_total):  5
len(GAP_total):  6
Seed : 31


C:\Users\gebre\AppData\Local\Temp\ipykernel_16812\2086752001.py:13: DtypeWarning: Columns (1,3,6,7,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{base_path}bipred_{seed}.csv")


len(GAP_total):  2
GAP_total: [[ 0.02804029  0.01897696  0.00387182  0.          0.0215358   0.01821331
   0.          0.01705278 -0.03661454  0.00360846  0.03818994  0.
   0.0046906  -0.087661  ]
 [ 0.         -0.00171406 -0.01531189  0.00306577  0.          0.
   0.02806835  0.          0.00263027  0.          0.04889882  0.00192993
   0.          0.        ]
 [-0.01849264 -0.01752791 -0.03099402  0.04269281 -0.03381096 -0.00333708
   0.06631837 -0.01851874  0.         -0.01790068  0.          0.03307649
  -0.0129928   0.03337618]
 [ 0.04116849  0.02752705  0.         -0.02471776  0.03815551 -0.00716548
  -0.10258822  0.00628646 -0.19573792  0.00325833 -0.09143969 -0.07710009
  -0.01368186 -0.21948096]
 [-0.00772148  0.          0.00761749 -0.13405797 -0.07810548  0.01536754
  -0.03579093 -0.07200972  0.15137835 -0.02668517 -0.05810636 -0.01368068
   0.06473829  0.06142496]]
len(GAP_total):  5
len(GAP_total):  6
Seed : 38


C:\Users\gebre\AppData\Local\Temp\ipykernel_16812\2086752001.py:13: DtypeWarning: Columns (1,3,6,7,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{base_path}bipred_{seed}.csv")


len(GAP_total):  2
GAP_total: [[ 0.02804029  0.01897696  0.00387182  0.          0.0215358   0.01821331
   0.          0.01705278 -0.03661454  0.00360846  0.03818994  0.
   0.0046906  -0.087661  ]
 [ 0.         -0.00171406 -0.01531189  0.00306577  0.          0.
   0.02806835  0.          0.00263027  0.          0.04889882  0.00192993
   0.          0.        ]
 [-0.01849264 -0.01752791 -0.03099402  0.04269281 -0.03381096 -0.00333708
   0.06631837 -0.01851874  0.         -0.01790068  0.          0.03307649
  -0.0129928   0.03337618]
 [ 0.04116849  0.02752705  0.         -0.02471776  0.03815551 -0.00716548
  -0.10258822  0.00628646 -0.19573792  0.00325833 -0.09143969 -0.07710009
  -0.01368186 -0.21948096]
 [-0.00772148  0.          0.00761749 -0.13405797 -0.07810548  0.01536754
  -0.03579093 -0.07200972  0.15137835 -0.02668517 -0.05810636 -0.01368068
   0.06473829  0.06142496]]
len(GAP_total):  5
len(GAP_total):  6
Seed : 47


C:\Users\gebre\AppData\Local\Temp\ipykernel_16812\2086752001.py:13: DtypeWarning: Columns (1,3,6,7,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{base_path}bipred_{seed}.csv")


len(GAP_total):  2
GAP_total: [[ 0.02804029  0.01897696  0.00387182  0.          0.0215358   0.01821331
   0.          0.01705278 -0.03661454  0.00360846  0.03818994  0.
   0.0046906  -0.087661  ]
 [ 0.         -0.00171406 -0.01531189  0.00306577  0.          0.
   0.02806835  0.          0.00263027  0.          0.04889882  0.00192993
   0.          0.        ]
 [-0.01849264 -0.01752791 -0.03099402  0.04269281 -0.03381096 -0.00333708
   0.06631837 -0.01851874  0.         -0.01790068  0.          0.03307649
  -0.0129928   0.03337618]
 [ 0.04116849  0.02752705  0.         -0.02471776  0.03815551 -0.00716548
  -0.10258822  0.00628646 -0.19573792  0.00325833 -0.09143969 -0.07710009
  -0.01368186 -0.21948096]
 [-0.00772148  0.          0.00761749 -0.13405797 -0.07810548  0.01536754
  -0.03579093 -0.07200972  0.15137835 -0.02668517 -0.05810636 -0.01368068
   0.06473829  0.06142496]]
len(GAP_total):  5
len(GAP_total):  6
Seed : 77
